这段代码的作用是使用深度学习模型 VGG16 提取图像的多个可能的语义描述，并使用预训练的自然语言处理模型（DistilBERT）对这些语义描述进行情感分类。

具体步骤如下：

1. 加载 VGG16 模型和预训练权重。VGG16 是一个用于图像分类的经典卷积神经网络模型。

2. 加载 DistilBERT 情感分类模型和对应的 tokenizer。这个模型在 SST-2 数据集上进行了微调，用于进行情感分类。

3. 定义一个函数 `extract_semantic_description()`，该函数接受一个图像文件路径作为输入，使用 VGG16 模型对图像进行预测，提取前几个最可能的类别作为图像的语义描述，并返回这些描述。

4. 指定要处理的图像文件路径，调用 `extract_semantic_description()` 函数获取图像的多个语义描述。

5. 将获取的多个语义描述组合成一个句子，前面加上 "A picture of "，以便进行情感分类。

6. 使用情感分析管道对组合的语义描述进行情感分类，返回结果。

通过将图像的语义信息和情感信息结合在一起，提供了一种从视觉内容到情感倾向的转换方法，可以帮助理解图像的情感含义。

In [39]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# 加载 VGG16 模型（包括顶层）
# 指定本地权重文件路径
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'

# 加载预训练的 VGG16 模型
model = VGG16(weights=weights_path, include_top=True)

# 加载图像并提取前几个最可能的类别及其概率作为语义描述
def extract_semantic_description(img_path, top_k=5):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = model.predict(x)
    # 获取前几个最可能的类别及其概率
    predicted_classes = decode_predictions(predictions, top=top_k)[0]
    semantic_descriptions = [(class_name, class_probability) for (_, class_name, class_probability) in predicted_classes]
    return semantic_descriptions

# 图像文件路径
img_path = 'cat.jpg'

# 提取前几个最可能的类别及其概率语义
semantic_descriptions = extract_semantic_description(img_path)
print("Top Semantic Descriptions:")
for class_name, class_probability in semantic_descriptions:
    print(f"{class_name}: {class_probability}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
Top Semantic Descriptions:
bath_towel: 0.49837571382522583
Egyptian_cat: 0.07177896052598953
tabby: 0.04368782788515091
Chihuahua: 0.026950838044285774
Boston_bull: 0.020928602665662766


In [37]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# 模型和tokenizer的名称或路径
model_name_or_path = './model/distilbert-base-uncased-finetuned-sst-2-english'

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# 加载 TensorFlow 版本的情感分类模型
model = TFAutoModelForSequenceClassification.from_pretrained(model_name_or_path)

# 预处理文本
text = "I love this product!"
inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True)

# 使用模型进行预测
outputs = model(inputs)

# 获取预测结果
predictions = tf.nn.softmax(outputs.logits, axis=-1)

# 将结果转换为情感标签
sentiment_label = "positive" if predictions[0][1] > predictions[0][0] else "negative"

# 打印情感分类结果
print("Sentiment:", sentiment_label)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Sentiment: positive


In [36]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

# 加载 VGG16 模型（包括顶层）
# 指定本地权重文件路径
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'

# 加载预训练的 VGG16 模型
model = VGG16(weights=weights_path, include_top=True)


# 指定本地模型目录路径
model_name_or_path = './model/distilbert-base-uncased-finetuned-sst-2-english'
# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# 加载 TensorFlow 版本的情感分类模型
model_distilbert = TFAutoModelForSequenceClassification.from_pretrained(model_name_or_path)
# 创建情感分析管道
sentiment_analysis = pipeline("sentiment-analysis", model=model_distilbert, tokenizer=tokenizer)


# 加载图像并提取最可能的类别作为语义描述
def extract_semantic_description(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = model.predict(x)
    # 获取最可能的类别索引
    predicted_class_index = np.argmax(predictions[0])
    # 获取类别名称
    predicted_class = decode_predictions(predictions, top=1)[0][0]
    return predicted_class[1]

# 图像文件路径
img_path = 'cat.jpg'

# 提取语义描述
semantic_description = extract_semantic_description(img_path)
print(f"Semantic Description: {semantic_description}")

# 对语义描述进行情感分类
sentiment_result = sentiment_analysis(f"A picture of a {semantic_description}")
print("Sentiment Analysis Result:", sentiment_result)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step
Semantic Description: bath_towel
Sentiment Analysis Result: [{'label': 'POSITIVE', 'score': 0.6200490593910217}]


提取图片的多个语义，将多个语义情感分类

In [40]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

# 加载 VGG16 模型（包括顶层）
# 指定本地权重文件路径
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'

# 加载预训练的 VGG16 模型
model = VGG16(weights=weights_path, include_top=True)

# 指定本地模型目录路径
model_name_or_path = './model/distilbert-base-uncased-finetuned-sst-2-english'
# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# 加载 TensorFlow 版本的情感分类模型
model_distilbert = TFAutoModelForSequenceClassification.from_pretrained(model_name_or_path)
# 创建情感分析管道
sentiment_analysis = pipeline("sentiment-analysis", model=model_distilbert, tokenizer=tokenizer)

# 加载图像并提取前几个最可能的类别及其概率作为语义描述
def extract_semantic_description(img_path, top_k=5):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = model.predict(x)
    # 获取前几个最可能的类别及其概率
    predicted_classes = decode_predictions(predictions, top=top_k)[0]
    semantic_descriptions = [class_name for (_, class_name, _) in predicted_classes]
    return semantic_descriptions

# 图像文件路径
img_path = 'cat.jpg'

# 提取多个语义描述
semantic_descriptions = extract_semantic_description(img_path)
print("Semantic Descriptions:", semantic_descriptions)

# 对每个语义描述进行情感分类
sentiment_results = []
for semantic_description in semantic_descriptions:
    # 对语义描述进行情感分类
    sentiment_result = sentiment_analysis(f"A picture of a {semantic_description}")
    sentiment_results.append(sentiment_result)

# 打印每个语义描述的情感分类结果
print("Semantic Description - Sentiment Analysis Result:")
for semantic_description, sentiment_result in zip(semantic_descriptions, sentiment_results):
    print(f"{semantic_description}: {sentiment_result}")


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step
Semantic Descriptions: ['bath_towel', 'Egyptian_cat', 'tabby', 'Chihuahua', 'Boston_bull']
Semantic Description - Sentiment Analysis Result:
bath_towel: [{'label': 'POSITIVE', 'score': 0.6200490593910217}]
Egyptian_cat: [{'label': 'POSITIVE', 'score': 0.993221640586853}]
tabby: [{'label': 'POSITIVE', 'score': 0.9991317391395569}]
Chihuahua: [{'label': 'POSITIVE', 'score': 0.9968589544296265}]
Boston_bull: [{'label': 'POSITIVE', 'score': 0.9845955967903137}]


将图片的多个语义组成一个句子，对句子情感分类

In [41]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

# 加载 VGG16 模型（包括顶层）
# 指定本地权重文件路径
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'

# 加载预训练的 VGG16 模型
model = VGG16(weights=weights_path, include_top=True)

# 指定本地模型目录路径
model_name_or_path = './model/distilbert-base-uncased-finetuned-sst-2-english'
# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# 加载 TensorFlow 版本的情感分类模型
model_distilbert = TFAutoModelForSequenceClassification.from_pretrained(model_name_or_path)
# 创建情感分析管道
sentiment_analysis = pipeline("sentiment-analysis", model=model_distilbert, tokenizer=tokenizer)

# 加载图像并提取前几个最可能的类别及其概率作为语义描述
def extract_semantic_description(img_path, top_k=5):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = model.predict(x)
    # 获取前几个最可能的类别及其概率
    predicted_classes = decode_predictions(predictions, top=top_k)[0]
    semantic_descriptions = [class_name for (_, class_name, _) in predicted_classes]
    return semantic_descriptions

# 图像文件路径
img_path = 'cat.jpg'

# 提取多个语义描述
semantic_descriptions = extract_semantic_description(img_path)
print("Semantic Descriptions:", semantic_descriptions)

# 将多个语义描述组合成一个句子
combined_description = "A picture of " + ", ".join(semantic_descriptions)

# 对组合的语义描述进行情感分类
sentiment_result = sentiment_analysis(combined_description)
print("Combined Semantic Description - Sentiment Analysis Result:", sentiment_result)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
Semantic Descriptions: ['bath_towel', 'Egyptian_cat', 'tabby', 'Chihuahua', 'Boston_bull']
Combined Semantic Description - Sentiment Analysis Result: [{'label': 'NEGATIVE', 'score': 0.779617190361023}]
